# Train rt-detr to detect weeds in potato fields

In this notebook we'll train a [rt-detr](https://docs.ultralytics.com/models/rtdetr/) model to detect weeds in a potato fields. 
The target weeds are:
- kikuyo
- lengua de vaca
- diente de león

These weeds are common in potatoes fields. The images was captured in 9-10 meters height with a drone. Then, the images was selected and cropped 
to 1920x1920 pixels. Finally, the images was labeled with Roboflow.

## Install dependencies
First of all, we'll check the GPU, and install the dependencies.

In [ ]:
!nvidia-smi
!pip install -q coco-eval pycocotools supervision ultralytics wandb

## Initialize weights & biases
We'll use weights & biases to log the training process.

In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient

project_name = 'rt-detr-potatoes'
user_secrets = UserSecretsClient()
key = user_secrets.get_secret('wandb')


wandb.login(key=key)
wandb.init(project=project_name, job_type='training')

# Create data.yaml
We'll create a data.yaml file with the classes and the paths to the images.

In [ ]:
import yaml

data_yaml = '''
path: /kaggle/input/potatoes-weeds-dataset/potatoes-dataset.v1-dataset-without-diente-leon.yolov8-obb

train: train/images
val: valid/images
test: test/images

names:
  0: diente_leon
  1: kikuyo
  2: lengua_vaca
  3: papa
'''

data = yaml.safe_load(data_yaml)

with open('data.yaml', 'w') as file:
    yaml.dump(data, file)

## Prepare model
We'll download the rt-detr model and prepare it before training.

In [ ]:
from ultralytics import RTDETR
from wandb.integration.ultralytics import add_wandb_callback


# Load a COCO-pretrained RT-DETR-l model
model = RTDETR('rtdetr-l.pt')

# Display model information (optional)
model.info()

# Add W&B Callback for Ultralytics
add_wandb_callback(model, enable_model_checkpointing=True)

## Train the model
We'll train the model.

In [ ]:
results = model.train(project=project_name,data='data.yaml', epochs=50, imgsz=640, device=[0, 1], batch=16)
results = model.val()

wandb.finish()

## Save the model
We'll save the model in the output folder.

In [ ]:
model.export()